## Import Library

In [1]:
import pandas as pd
import uuid
import requests
import io

In [2]:
df = pd.read_parquet('fhvhv_tripdata_2024-09.parquet')

In [6]:
#Sample 100,000 records
df = df.sample(n=100000, random_state=1).reset_index(drop=True)
df.to_parquet('fhvhv_tripdata_2024-09_trim.parquet', engine='fastparquet')

In [7]:
df

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2024-09-09 18:25:19,2024-09-09 18:26:47,2024-09-09 18:26:52,2024-09-09 18:30:38,216,132,1.010,...,1.03,0.00,2.5,0.0,5.39,N,N,N,N,N
1,HV0005,B03406,None,2024-09-10 08:53:34,NaT,2024-09-10 08:59:00,2024-09-10 09:08:01,196,82,1.467,...,1.03,0.00,0.0,0.0,7.27,N,N,N,N,N
2,HV0003,B03404,B03404,2024-09-01 18:08:39,2024-09-01 18:14:49,2024-09-01 18:16:40,2024-09-01 18:29:10,144,90,1.830,...,1.64,2.75,0.0,0.0,10.07,N,N,N,N,N
3,HV0003,B03404,B03404,2024-09-13 11:59:30,2024-09-13 12:01:29,2024-09-13 12:03:07,2024-09-13 12:34:13,49,145,6.960,...,2.34,0.00,0.0,0.0,27.60,N,N,N,N,N
4,HV0003,B03404,B03404,2024-09-19 05:08:25,2024-09-19 05:10:38,2024-09-19 05:11:08,2024-09-19 05:16:14,139,203,1.550,...,0.79,0.00,0.0,0.0,5.40,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,HV0005,B03406,None,2024-09-03 21:49:06,NaT,2024-09-03 21:52:26,2024-09-03 22:13:16,231,239,5.510,...,2.30,2.75,0.0,0.0,19.71,N,N,N,N,N
99996,HV0003,B03404,B03404,2024-09-29 01:50:25,2024-09-29 01:50:58,2024-09-29 01:53:00,2024-09-29 01:58:43,52,33,1.070,...,0.65,0.00,0.0,0.0,7.67,N,N,N,N,N
99997,HV0003,B03404,B03404,2024-09-26 03:03:54,2024-09-26 03:07:48,2024-09-26 03:08:05,2024-09-26 03:15:06,42,41,1.420,...,0.95,0.00,0.0,0.0,6.02,N,N,N,N,N
99998,HV0003,B03404,B03404,2024-09-24 21:20:20,2024-09-24 21:24:45,2024-09-24 21:25:01,2024-09-24 21:32:51,206,206,1.920,...,0.87,0.00,0.0,0.0,7.17,N,N,N,N,N


In [4]:
df.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19209788 entries, 0 to 19209787
Data columns (total 24 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[us]
 4   on_scene_datetime     datetime64[us]
 5   pickup_datetime       datetime64[us]
 6   dropoff_datetime      datetime64[us]
 7   PULocationID          int32         
 8   DOLocationID          int32         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64       
 19

In [8]:
# Set primary key for fact_table
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.apply(lambda x: uuid.uuid4().hex, axis=1)

In [9]:
#Create datetime_dim table
datetime_dim = df[['trip_id','request_datetime','on_scene_datetime','pickup_datetime','dropoff_datetime']].reset_index(drop=True)
datetime_dim['request_datetime'] = datetime_dim['request_datetime']
datetime_dim['request_hour'] = datetime_dim['request_datetime'].dt.hour
datetime_dim['request_day'] = datetime_dim['request_datetime'].dt.day
datetime_dim['request_month'] = datetime_dim['request_datetime'].dt.month
datetime_dim['request_year'] = datetime_dim['request_datetime'].dt.year
datetime_dim['request_weekday'] = datetime_dim['request_datetime'].dt.weekday

datetime_dim['on_scene_datetime']= datetime_dim['on_scene_datetime']
datetime_dim['on_scene_hour']=datetime_dim['on_scene_datetime'].dt.hour
datetime_dim['on_scene_day']=datetime_dim['on_scene_datetime'].dt.day
datetime_dim['on_scene_month']=datetime_dim['on_scene_datetime'].dt.month
datetime_dim['on_scene_year']=datetime_dim['on_scene_datetime'].dt.year
datetime_dim['on_scene_weekday']=datetime_dim['on_scene_datetime'].dt.weekday

datetime_dim['pickup_datetime']=datetime_dim['pickup_datetime']
datetime_dim['pickup_hour']=datetime_dim['pickup_datetime'].dt.hour
datetime_dim['pickup_day']=datetime_dim['pickup_datetime'].dt.day
datetime_dim['pickup_month']=datetime_dim['pickup_datetime'].dt.month
datetime_dim['pickup_year']=datetime_dim['pickup_datetime'].dt.year
datetime_dim['pickup_weekday']=datetime_dim['pickup_datetime'].dt.weekday

datetime_dim['dropoff_datetime']=datetime_dim['dropoff_datetime']
datetime_dim['dropoff_hour']=datetime_dim['dropoff_datetime'].dt.hour
datetime_dim['dropoff_day']=datetime_dim['dropoff_datetime'].dt.day
datetime_dim['dropoff_month']=datetime_dim['dropoff_datetime'].dt.month
datetime_dim['dropoff_year']=datetime_dim['dropoff_datetime'].dt.year
datetime_dim['dropoff_weekday']=datetime_dim['dropoff_datetime'].dt.weekday

datetime_dim = datetime_dim[['trip_id','request_datetime','request_hour','request_day','request_month','request_year','request_weekday','on_scene_datetime','on_scene_hour','on_scene_day','on_scene_month','on_scene_year','on_scene_weekday','pickup_datetime','pickup_hour','pickup_day','pickup_month','pickup_year','pickup_weekday','dropoff_datetime','dropoff_hour','dropoff_day','dropoff_month','dropoff_year','dropoff_weekday']]
datetime_dim.head()

#Create hvfhs_license_dim table
hvfhs_licence_code = {
    "HV0002":"Juno",
    "HV0003":"Uber",
    "HV0004":"Via",
    "HV0005":"Lyft"
}

hvfhs_license_dim = df[['hvfhs_license_num']].drop_duplicates().reset_index(drop=True)
hvfhs_license_dim['hvfhs_license_name']=hvfhs_license_dim['hvfhs_license_num'].map(hvfhs_licence_code)

#Create trip_dim table
trip_dim = df[['trip_id','trip_miles','trip_time','PULocationID','DOLocationID']].reset_index(drop=True)

#Create fare_dim table
fare_dim = df[['trip_id','base_passenger_fare','tolls','bcf','sales_tax','congestion_surcharge','airport_fee','tips','driver_pay']].reset_index(drop=True)

In [10]:
#Create fact_table

fact_table = df.merge(trip_dim, on = 'trip_id') \
            .merge(datetime_dim, on='trip_id') \
            .merge(hvfhs_license_dim, on='hvfhs_license_num') \
            .merge(fare_dim, on = 'trip_id') \
            [['trip_id', 'hvfhs_license_num', 'shared_request_flag','shared_match_flag','wav_request_flag','wav_match_flag','access_a_ride_flag','originating_base_num','dispatching_base_num']]

In [11]:
fact_table.head()

,trip_id,hvfhs_license_num,shared_request_flag,shared_match_flag,wav_request_flag,wav_match_flag,access_a_ride_flag,originating_base_num,dispatching_base_num
0,afeac25b59b04daea4b90829f26164bc,HV0003,N,N,N,N,N,B03404,B03404
1,711573dd4f74411cbed9e03ce988e8d0,HV0005,N,N,N,N,N,None,B03406
2,1e271dfd16a945629c7e7d75dd83a56b,HV0003,N,N,N,N,N,B03404,B03404
3,351fc96ee46d44d48096ac9160926d81,HV0003,N,N,N,N,N,B03404,B03404
4,a153e67357474e7bb0b7ed5b272e907e,HV0003,N,N,N,N,N,B03404,B03404


SSH command:

<!-- 1. Update Debian OS <br>
sudo apt-get update -y

2. Install Python & required dependancy <br>
sudo apt-get install python3-distutilssud <br>
sudo apt-get install python3-apt <br> -->
<!-- sudo apt install python3-pip -->

3. Install the virtual environment (To avoid conflicts with system packages from ubuntu):
sudo apt install python3-venv
python3 -m venv .venv
source .venv/bin/activate

4. Install Mage data pipeline & google cloud<br>
pip install mage-ai
In the VM, install google-cloud: sudo pip3 install google-cloud & sudo pip3 install google-cloud-bigquery

6. Run mage start uber-de-mark-project to start:<br>
mage start uber-de-mark <br>

This initialize my project with port 6789. <br>
But I also needed to configure the Firewall rule of my VM to allow public access (0.0.0.0/0). Then I can access the portal using the VM external IP with the port 6789 <br>
http://34.19.58.189:6789 (Note: This is my VM ip, so if you create another VM, you will use your VM ip)

7. In the load data step from Mage pipeline (default is pandas, but we use parquet)

url = 'https://storage.googleapis.com/uber-tlc-de-mark/fhvhv_tripdata_2024-09_trim.parquet'
response = requests.get(url)

with io.BytesIO(response.content) as file:
        df = pd.read_parquet(file)

8. After loading, in transformation layer, after processing the data, return the dictionary to load into BigQuery
{"datetime_dim":datetime_dim.to_dict(orient="dict"),
    "hvfhs_license_dim":hvfhs_license_dim.to_dict(orient="dict"),
    "trip_dim":trip_dim.to_dict(orient="dict"),
    "fare_dim":fare_dim.to_dict(orient="dict"),
    "fact_table":fact_table.to_dict(orient="dict")}

9. Create Exporter Layer (Load Layer) to Big Query:
- Require API & Service key to put in yaml file in mage
- 